In [2]:
import os

In [3]:
os.environ["SNOWFLAKE_USER"] = "RICHARDPERSONAL"
os.environ["SNOWFLAKE_PASSWORD"] = "Rp@976559SN"
os.environ["SNOWFLAKE_ACCOUNT"] = "mn34265.eu-central-1.aws"
os.environ["SNOWFLAKE_WAREHOUSE"] = "TEST_WH"

# Dask-Snowflake Integration Demo

## 1. Set-up Dask Resources

In [4]:
import coiled

In [ ]:
# coiled.create_software_environment(
#     name="snowflake",
#     account="coiled-examples",
#     pip=[
#         "dask[distributed, dataframe, diagnostics]==2021.11.2",
#         "snowflake-connector-python",
#         "dask-snowflake",
#         "lz4",
#         "xgboost",
#     ],
# )

In [5]:
# spin up cluster
cluster = coiled.Cluster(
    name="coiled-snowflake",
    software="coiled-examples/snowflake",
    n_workers=20,
    shutdown_on_close=False,
    scheduler_options={'idle_timeout':'2 hours'},
    backend_options={'spot':'True'},
)

/Users/rpelgrim/mambaforge/envs/snowflake/lib/python3.9/site-packages/rich/live.py:221: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-rrpelgr71-103275-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-rrpelgr71-103275-firewall -> coiled-dask-rrpelgr71-103275-firewall]
Created FW rules: coiled-dask-rrpelgr71-103275-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-rrpelgr71-103275-cluster-firewall -> coiled-dask-rrpelgr71-103275-cluster-firewall]
Created scheduler VM: coiled-dask-rrpelgr71-103275-scheduler (type: t3a.medium, ip: ['3.232.96.252'])


In [6]:
# connect cluster to Dask
from dask.distributed import Client
client = Client(cluster)
client.dashboard_link

/Users/rpelgrim/mambaforge/envs/snowflake/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| pandas  | 1.3.4  | 1.3.5     | 1.3.5   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


'http://3.232.96.252:8787'

## 2. Connect to Snowflake

In [7]:
import os
import snowflake.connector

In [8]:
ctx = snowflake.connector.connect(
    user=os.environ["SNOWFLAKE_USER"],
    password=os.environ["SNOWFLAKE_PASSWORD"],
    account=os.environ["SNOWFLAKE_ACCOUNT"],
)

MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

In [20]:
cs = ctx.cursor()

schema = "TPCDS_SF100TCL"
table = "CALL_CENTER"

cs.execute("USE SNOWFLAKE_SAMPLE_DATA")
cs.execute("SELECT * FROM " + schema + "." + table)

one_row = str(cs.fetchone())

print(one_row)

(1, 'AAAAAAAABAAAAAAA', datetime.date(1998, 1, 1), None, None, 2450952, 'NY Metro', 'large', 597159671, 481436415, '8AM-4PM', 'Bob Belcher', 6, 'More than other authori', 'Shared others could not count fully dollars. New members ca', 'Julius Tran', 3, 'pri', 6, 'cally', '730', 'Ash Hill', 'Boulevard', 'Suite 0', 'Georgetown', 'Harmon County', 'OK', '77057', 'United States', Decimal('-6.00'), Decimal('0.11'))


## 3. Parallel Write to Snowflake

In [21]:
import dask

ddf = dask.datasets.timeseries(
    start="2021-01-01",
    end="2021-03-31",
)

In [22]:
# create warehouse and database
cs.execute("CREATE WAREHOUSE IF NOT EXISTS dask_snowflake_wh")
cs.execute("CREATE DATABASE IF NOT EXISTS dask_snowflake_db")
cs.execute("USE DATABASE dask_snowflake_db")

In [23]:
from dask_snowflake import to_snowflake

In [24]:
connection_kwargs = {
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_PASSWORD"],
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
    "database": "dask_snowflake_db",
    "schema": "PUBLIC",
}

In [25]:
%%time
to_snowflake(
    ddf,
    name="dask_snowflake_table",
    connection_kwargs=connection_kwargs,
)

/home/ec2-user/mambaforge/envs/snowflake/lib/python3.9/site-packages/snowflake/sqlalchemy/snowdialect.py:206: SAWarning: Dialect snowflake:snowflake will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = connection.execute(


CPU times: user 936 ms, sys: 44.5 ms, total: 981 ms
Wall time: 1min 13s


## 4. Parallel Read from Snowflake

In [26]:
from dask_snowflake import read_snowflake

In [27]:
%%time
snowflake_data = read_snowflake(
    query="""
      SELECT *
      FROM dask_snowflake_table;
   """,
    connection_kwargs=connection_kwargs,
)

print(snowflake_data.head())

     ID      NAME         X         Y
0  1029   Norbert  0.652481 -0.937071
1   992     Laura  0.063575  0.909713
2  1002  Patricia  0.593139 -0.653950
3  1036       Dan -0.340827  0.678265
4  1042     Frank  0.052302  0.782666
CPU times: user 262 ms, sys: 7.89 ms, total: 270 ms
Wall time: 4.51 s


In [28]:
snowflake_data

,ID,NAME,X,Y
npartitions=74,,,,
,int16,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [29]:
result = snowflake_data.X.mean()
print(result.compute())

7.50774355061549e-05


## 5. Machine Learning

In [ ]:
SCHEMA = "SNOWFLAKE_SAMPLE_DATA.TPCH_SF100"

example_query=f"""
SELECT
    
    C_CUSTKEY,
    C_NAME,
    SUM(L_QUANTITY) AS sum_qty,
    SUM(PS_AVAILQTY) AS sum_avail_qty,
    MAX(P_RETAILPRICE) AS max_retail_price
    
    FROM {SCHEMA}.CUSTOMER
    
        JOIN {SCHEMA}.ORDERS
            ON C_CUSTKEY = O_CUSTKEY
            
            JOIN {SCHEMA}.LINEITEM
                ON L_ORDERKEY = O_ORDERKEY
                
                JOIN {SCHEMA}.PART
                    ON P_PARTKEY = L_PARTKEY
                    
                    JOIN {SCHEMA}.PARTSUPP
                        ON P_PARTKEY = PS_PARTKEY
    
    WHERE PS_SUPPLYCOST > 10

GROUP BY C_CUSTKEY, C_NAME
"""

In [ ]:
connection_kwargs = {
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_PASSWORD"],
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
    "database": "SNOWFLAKE_SAMPLE_DATA",
    "schema": "TPCH_SF100",
}

In [ ]:
%%time
ddf = read_snowflake(
    query=example_query,
    connection_kwargs=connection_kwargs,
)

In [ ]:
ddf.head()

In [ ]:
import xgboost as xgb